<a href="https://colab.research.google.com/github/ChenshuLiu/Video-region-specific-RGB-value-extraction/blob/main/Mechanochromic_SCL_Pressure_to_RGB_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Data import & Preprocessing

In [ ]:
RGB_pressure = pd.read_csv('/content/drive/MyDrive/Mechanochromic SCL/RGB_val.csv')
RGB_pressure_cleaned = RGB_pressure.dropna()
RGB_pressure_cleaned = RGB_pressure_cleaned.drop('File name', axis = 1)
RGB_pressure_cleaned.head(5)

,Red channel,Green channel,Blue Channel,Pressure
0,133.349545,126.564597,173.291129,0.0
2,129.457818,120.743558,171.032468,0.0
3,110.719875,104.619572,150.785944,0.0
5,111.222252,104.103066,152.429154,0.0
8,102.886972,97.231988,141.111307,0.0


# Training testing data prep

In [ ]:
RGB_val = RGB_pressure_cleaned.loc[:, ['Red channel', 'Green channel', 'Blue Channel']]
Pressure_val = RGB_pressure_cleaned.loc[:, 'Pressure']
X_train, X_test, y_train, y_test = train_test_split(RGB_val, Pressure_val, test_size=0.2, random_state=123)

# convert to tensors to fit the neural network
X_tensor = torch.tensor(RGB_val.values, dtype=torch.float32)
y_tensor = torch.tensor(Pressure_val.values, dtype=torch.float32)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

print("X_train shape:", X_train_tensor.shape)
print("y_train shape:", y_train_tensor.shape)
print("X_test shape:", X_test_tensor.shape)
print("y_test shape:", y_test_tensor.shape)

X_train shape: torch.Size([138, 3])
y_train shape: torch.Size([138])
X_test shape: torch.Size([35, 3])
y_test shape: torch.Size([35])


# Simple neural network construction
Mapping (1, 3) input to single numerical output

In [34]:
# Define the neural network
class P_RGBNet(nn.Module):
    def __init__(self):
        super(P_RGBNet, self).__init__()
        self.fc1 = nn.Linear(3, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Instantiate the model, loss function, and optimizer
model = P_RGBNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 500
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_tensor)

    # Calculate the loss
    loss = criterion(outputs.squeeze(), y_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every 10 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        torch.save(model.state_dict(), '/content/drive/MyDrive/Mechanochromic SCL/Model Log/P_RGBNet_e{}.pt'.format(epoch+1))

Epoch [100/500], Loss: 0.3743
Epoch [200/500], Loss: 0.3355
Epoch [300/500], Loss: 0.3083
Epoch [400/500], Loss: 0.2914
Epoch [500/500], Loss: 0.2819


In [ ]:
# Example prediction
X_test = torch.tensor([[80, 80, 170]], dtype=torch.float32)
prediction = model(X_test)
print("Prediction:", prediction.item())

Prediction: 1.6724176406860352
